In [1]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import statsmodels.api as sm
import joblib
import json
import os
from io import StringIO


In [3]:
# see a few json files 
arr = os.listdir('../retroData')
arr = [i for i in arr if i.endswith('json')]
arr[0:5]

['IBV_116891.json',
 'IBV_65258.json',
 'IBV_44155.json',
 'IBV_2152.json',
 'IBV_48875.json']

### Transfer the py file here in the notebook chunk (current version is still Anson's notebook#2)

In [13]:
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import joblib
import json
import sys
import logging
def ndmodeling(
    jsonbody,
    modelfilepath="src/ND_Model/NegativeDB_model_V3_Testbed.pkl",
    datacleanfilepath="/Users/starsrain/2025_concord/loonie_rebuild_2025/ndb_retrain_notebooks/dataclean_nb_fillna.csv"
):
    logging.basicConfig(filename="NDModel_WarningErrorLog_2025.log", level=logging.WARNING, 
            format="%(asctime)s - %(levelname)s - %(message)s")
    stepname = "1 Loading Json"

    try:
        if type(jsonbody) is str:
            nb_dict = json.loads(jsonbody)
        else:
            nb_dict = jsonbody
        if nb_dict["NDB"]["results"] == []:
            return '{"ModelScore":999,"NDBand":' + str(6) + "}"
        jsonString = json.dumps(nb_dict["NDB"]["results"])
        df_ = pd.DataFrame(json.loads(jsonString))
        df_["accountnumber"] = nb_dict["NDB"]["accountnumber"]

        # Data Cleaning pre feature engineering
        stepname = "Data Cleaning pre feature engineering"
        df_["amount"] = df_["amount"].replace("", 0).astype(float)
        df_["requestDate"] = pd.to_datetime(
        df_["requestDate"], format="%Y-%m-%dT%H:%M:%S.%fZ", utc=True, errors="coerce"
)
        df_["Yeardiff"] = df_.apply(lambda x: (date.today().year - x["requestDate"].date().year), axis=1)
        df_["Monthdiff"] = df_.apply(lambda x: (date.today().month - x["requestDate"].date().month), axis=1)
        df_["Weekdiff"] = df_.apply(
            lambda x: (date.today().isocalendar()[1] - x["requestDate"].isocalendar()[1]),
            axis=1,
        )
        df_["Daydiff"] = df_.apply(lambda x: (date.today() - x["requestDate"].date()).days, axis=1)
        df_["phone1"] = df_["phone1"].fillna("")
        df_["phone2"] = df_["phone2"].fillna("")

        # Feature engineering / feature generation
        stepname = "2 Feature Generation"
        frequency = df_.groupby("accountnumber").size().reset_index()
        frequency.columns = ["accountnumber", "frequency"]

        Loan = df_.groupby("accountnumber")["amount"].agg(["mean", "median", "std"])
        Loan.columns = ["avg_amountReq", "med_amountReq", "std_amountReq"]
        Loan = Loan.reset_index()

        tc_date = df_.groupby("accountnumber")[["Yeardiff", "Monthdiff", "Weekdiff", "Daydiff"]].agg(
            ["max", "min", "mean", "median"]
        )
        tc_date.columns = [
            "Yeardiff_max",
            "Yeardiff_min",
            "Yeardiff_mean",
            "Yeardiff_median",
            "Monthdiff_max",
            "Monthdiff_min",
            "Monthdiff_mean",
            "Monthdiff_median",
            "Weekdiff_max",
            "Weekdiff_min",
            "Weekdiff_mean",
            "Weekdiff_median",
            "Daydiff_max",
            "Daydiff_min",
            "Daydiff_mean",
            "Daydiff_median",
        ]
        tc_date = tc_date.reset_index()

        df_["phone1"] = (
            df_.phone1.astype(str).str.replace(r"[\(\)-]", "", regex=True).str.replace(r"\s", "", regex=True)
        )
        df_["phone2"] = (
            df_.phone2.astype(str).str.replace(r"[\(\)-]", "", regex=True).str.replace(r"\s", "", regex=True)
        )

        df_["cust_cell_number"] = df_.phone1.str.extract("(\d+)").astype(str)
        df_["cust_phone_number"] = df_.phone2.str.extract("(\d+)").astype(str)

        df_["true_phone"] = df_.apply(
            lambda x: 0 if ((len(x["cust_phone_number"]) != 10) & (len(x["cust_cell_number"]) != 10)) else 1,
            axis=1,
        )
        df_["cust_cell_number"] = df_.phone1.apply(lambda x: "nan" if len(x) != 10 else x)
        df_["cust_phone_number"] = df_.phone2.apply(lambda x: "nan" if len(x) != 10 else x)
        df_["phone_cell"] = df_.apply(
            lambda x: 0 if x["cust_phone_number"] == x["cust_cell_number"] else 1,
            axis=1,
        )

        phone = df_.groupby("accountnumber")["true_phone"].agg(["max", "sum", "count"])
        phone["correct_phone_rate"] = phone["sum"] / phone["count"]
        phone.columns = [
            "have_valid_phone",
            "times_valid_phone",
            "total_phone_enter",
            "correct_phone_rate",
        ]
        phone = phone.reset_index()

        phone_count = df_[(df_["true_phone"] == 1) & (df_["cust_cell_number"] != "nan")]
        phone_count = pd.DataFrame(
            phone_count.groupby("accountnumber")[["cust_phone_number", "cust_cell_number"]].apply(
                lambda x: pd.unique(x.values.ravel()).tolist()
            )
        )
        phone_count.columns = ["phone_list"]
        phone_count["num_unique_valid_phone"] = phone_count.phone_list.apply(
            lambda x: len(x) if "nan" not in x else len(x) - 1
        )
        phone_count = phone_count.reset_index()

        phone_features = pd.merge(
            phone,
            phone_count[["accountnumber", "num_unique_valid_phone"]],
            on="accountnumber",
            how="left",
        )
        phone_features["num_unique_valid_phone"] = phone_features["num_unique_valid_phone"].fillna(0)

        df_["refused"] = (df_["status"] == "refused").astype(int)
        df_["fraudster"] = (df_["status"] == "fraudster").astype(int)
        df_["duplicates"] = (df_["status"] == "duplicates").astype(int)
        df_["in-collection"] = (df_["status"] == "in-collection").astype(int)
        df_["loan-pay-in-full"] = (df_["status"] == "loan-pay-in-full").astype(int)

        ### Make sure you change it!!!
        curdate = date.today()
        curdate = df_["requestDate"].max().date()  # remove it before deployment
        df_["within_last_30day"] = df_.apply(lambda x: (curdate - x["requestDate"].date()).days <= 30, axis=1)

        loanspaidoff_count = df_.groupby("accountnumber").apply(lambda x: x["loan-pay-in-full"].sum()).reset_index()
        incollection_count = df_.groupby("accountnumber").apply(lambda x: x["in-collection"].sum()).reset_index()
        try:
            loanspaidoff_count_in30days = (
                df_.groupby("accountnumber")
                .apply(lambda x: x[x.within_last_30day == 1]["loan-pay-in-full"].sum())
                .reset_index()
            )
            incollection_count_in30days = (
                df_.groupby("accountnumber")
                .apply(lambda x: x[x.within_last_30day == 1]["in-collection"].sum())
                .reset_index()
            )
        except:
            loanspaidoff_count_in30days = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Loanspaidoff_count_in30days": [0],
                }
            )
            incollection_count_in30days = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Incollection_count_in30days": [0],
                }
            )
        loanspaidoff_rate = (
            df_.groupby("accountnumber")
            .apply(
                lambda x: x["loan-pay-in-full"].sum()
                / (x["loan-pay-in-full"].sum() + x["in-collection"].sum() + 0.00001)
            )
            .reset_index()
        )
        loanspaidoff_count.columns = ["accountnumber", "Loanspaidoff_count"]
        incollection_count.columns = ["accountnumber", "Incollection_count"]
        loanspaidoff_count_in30days.columns = [
            "accountnumber",
            "Loanspaidoff_count_in30days",
        ]
        incollection_count_in30days.columns = [
            "accountnumber",
            "Incollection_count_in30days",
        ]
        loanspaidoff_rate.columns = ["accountnumber", "Loanspaidoff_rate"]

        fraudster_app_count = df_.groupby("accountnumber").apply(lambda x: x["fraudster"].sum()).reset_index()
        fraudster_lender_count = (
            df_.groupby("accountnumber").apply(lambda x: x[x.fraudster == 1]["lender"].nunique()).reset_index()
        )
        if len(df_[df_.within_last_30day == True]) == 0:
            fraudster_app_count_in30days = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Fraudster_app_count_in30days": [0],
                }
            )
            fraudster_lender_count_in30days = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Fraudster_lender_count_in30days": [0],
                }
            )
        else:
            fraudster_app_count_in30days = (
                df_.groupby("accountnumber")
                .apply(lambda x: x[x.within_last_30day == 1]["fraudster"].sum())
                .reset_index()
            )
            fraudster_lender_count_in30days = (
                df_.groupby("accountnumber")
                .apply(lambda x: x[(x.within_last_30day == 1) & (x.fraudster == 1)]["lender"].nunique())
                .reset_index()
            )

        fraudster_app_count.columns = ["accountnumber", "Fraudster_app_count"]
        fraudster_lender_count.columns = ["accountnumber", "Fraudster_lender_count"]
        fraudster_app_count_in30days.columns = [
            "accountnumber",
            "Fraudster_app_count_in30days",
        ]
        fraudster_lender_count_in30days.columns = [
            "accountnumber",
            "Fraudster_lender_count_in30days",
        ]

        refused_count = df_.groupby("accountnumber").apply(lambda x: x["refused"].sum()).reset_index()
        refused_rate = (
            df_.groupby("accountnumber")
            .apply(
                lambda x: x["refused"].sum() / (x["refused"].count() - x["duplicates"].sum())
                if (x["refused"].count() - x["duplicates"].sum()) != 0
                else 0
            )
            .reset_index()
        )
        refused_count.columns = ["accountnumber", "Refused_count"]
        refused_rate.columns = ["accountnumber", "Refused_rate"]

        if len(df_[df_.within_last_30day == 1]) == 0:
            refused_count_within30day = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Refused_count_within30days": [0],
                }
            )
            refused_rate_within30day = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Refused_rate_within30days": [0],
                }
            )
        else:
            refused_count_within30day = (
                df_[df_.within_last_30day == True]
                .groupby("accountnumber")
                .apply(lambda row: row["refused"].sum())
                .reset_index()
            )
            refused_rate_within30day = (
                df_[df_.within_last_30day == True]
                .groupby("accountnumber")
                .apply(
                    lambda row: row["refused"].sum() / (row["refused"].count() - row["duplicates"].sum())
                    if (row["refused"].count() - row["duplicates"].sum()) != 0
                    else 0
                )
                .reset_index()
            )
        if len(df_[df_.within_last_30day == 0]) == 0:
            refused_count_before30day = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Refused_count_before30days": [0],
                }
            )
            refused_rate_before30day = pd.DataFrame(
                {
                    "accountnumber": [nb_dict["NDB"]["accountnumber"]],
                    "Refused_rate_before30days": [0],
                }
            )
        else:
            refused_count_before30day = (
                df_[df_.within_last_30day == False]
                .groupby("accountnumber")
                .apply(lambda row: row["refused"].sum())
                .reset_index()
            )
            refused_rate_before30day = (
                df_[df_.within_last_30day == False]
                .groupby("accountnumber")
                .apply(
                    lambda row: row["refused"].sum() / (row["refused"].count() - row["duplicates"].sum())
                    if (row["refused"].count() - row["duplicates"].sum()) != 0
                    else 0
                )
                .reset_index()
            )

        try:
            refused_count_within30day.columns = [
                "accountnumber",
                "Refused_count_within30days",
            ]
            refused_rate_within30day.columns = [
                "accountnumber",
                "Refused_rate_within30days",
            ]
        except:
            refused_count_within30day = pd.DataFrame(
                refused_count_before30day.values,
                columns=["accountnumber", "Refused_count_within30days"],
            )
            refused_rate_within30day = pd.DataFrame(
                refused_rate_before30day.values,
                columns=["accountnumber", "Refused_rate_within30days"],
            )
        try:
            refused_count_before30day.columns = [
                "accountnumber",
                "Refused_count_before30days",
            ]
            refused_rate_before30day.columns = [
                "accountnumber",
                "Refused_rate_before30days",
            ]
        except:
            refused_count_before30day = pd.DataFrame(
                refused_count_within30day.values,
                columns=["accountnumber", "Refused_count_before30days"],
            )
            refused_rate_before30day = pd.DataFrame(
                refused_rate_within30day.values,
                columns=["accountnumber", "Refused_rate_before30days"],
            )

        status_summary = loanspaidoff_count
        for subdf in [
            incollection_count,
            loanspaidoff_count_in30days,
            incollection_count_in30days,
            loanspaidoff_rate,
            fraudster_app_count,
            fraudster_lender_count,
            fraudster_app_count_in30days,
            fraudster_lender_count_in30days,
            refused_count,
            refused_rate,
            refused_count_within30day,
            refused_rate_within30day,
            refused_count_before30day,
            refused_rate_before30day,
        ]:
            status_summary = status_summary.merge(subdf, on="accountnumber", how="left")

        feature_df = frequency
        for d in [Loan, tc_date, phone_features, status_summary]:
            feature_df = pd.merge(feature_df, d, on="accountnumber", how="outer")

        # ND Data Cleaning post feature engineering
        stepname = "3 Data Cleaning Post Feature Gen"
        value_to_fillnull = pd.read_csv(datacleanfilepath)  # 'saved/dataclean_nb_fillna.csv'

        cols = [
            "avg_amountReq",
            "med_amountReq",
            "std_amountReq",
            "times_valid_phone",
            "total_phone_enter",
            "correct_phone_rate",
            "num_unique_valid_phone",
            "Yeardiff_max",
            "Yeardiff_min",
            "Yeardiff_mean",
            "Yeardiff_median",
            "Monthdiff_max",
            "Monthdiff_min",
            "Monthdiff_mean",
            "Monthdiff_median",
            "Weekdiff_max",
            "Weekdiff_min",
            "Weekdiff_mean",
            "Weekdiff_median",
            "Daydiff_max",
            "Daydiff_min",
            "Daydiff_mean",
            "Daydiff_median",
        ]
        feature_df[cols] = feature_df[cols].fillna(value_to_fillnull.loc[0, cols])

        # Model loading and scoring
        stepname = "4 Model Loading and Scoring"

        """ clf_nb = joblib.load(modelfilepath)  # example: datacleanfilepath = "saved/NegativeDB_model.pkl"

        features_nb = [
            "frequency",
            "avg_amountReq",
            "med_amountReq",
            "std_amountReq",
            "Yeardiff_max",
            "Yeardiff_min",
            "Yeardiff_mean",
            "Yeardiff_median",
            "Monthdiff_max",
            "Monthdiff_min",
            "Monthdiff_mean",
            "Monthdiff_median",
            "Weekdiff_max",
            "Weekdiff_min",
            "Weekdiff_mean",
            "Weekdiff_median",
            "Daydiff_max",
            "Daydiff_min",
            "Daydiff_median",
            "Daydiff_mean",
            "have_valid_phone",
            "times_valid_phone",
            "total_phone_enter",
            "correct_phone_rate",
            "num_unique_valid_phone",
            "Loanspaidoff_count",
            "Incollection_count",
            "Loanspaidoff_count_in30days",
            "Incollection_count_in30days",
            "Loanspaidoff_rate",
            "Fraudster_app_count",
            "Fraudster_lender_count",
            "Fraudster_app_count_in30days",
            "Fraudster_lender_count_in30days",
            "Refused_count",
            "Refused_rate",
            "Refused_count_within30days",
            "Refused_rate_within30days",
            "Refused_count_before30days",
            "Refused_rate_before30days",
        ]

        # Output 1: Prediction of FPD First Attempt
        feature_df["NDScore"] = 1000 - (clf_nb.predict_proba(feature_df[features_nb])[:, 1] * 1000).astype(int)
        # Output 2: Model Band
        feature_df["NDBand"] = np.where(
            feature_df["NDScore"] < 376,
            1,
            np.where(
                feature_df["NDScore"] < 580,
                2,
                np.where(
                    feature_df["NDScore"] < 688,
                    3,
                    np.where(feature_df["NDScore"] < 766, 4, 5),
                ),
            ),
        ) """

    except:
        try:
            result = {"AccountID": str(nb_dict["NDB"]["accountnumber"]), "ErrorInStep": stepname}
        except:
            result = {"AccountID": "Not Available", "ErrorInStep": stepname}
        finally:
            return result

  
    return feature_df


### Check to see if model works as intended for one json input file

In [14]:
data_input_path = '../retroData/'+str('IBV_126164.json')
with open(data_input_path) as f:
    data = f.read()
x = ndmodeling(data)
x.columns

Index(['accountnumber', 'frequency', 'avg_amountReq', 'med_amountReq',
       'std_amountReq', 'Yeardiff_max', 'Yeardiff_min', 'Yeardiff_mean',
       'Yeardiff_median', 'Monthdiff_max', 'Monthdiff_min', 'Monthdiff_mean',
       'Monthdiff_median', 'Weekdiff_max', 'Weekdiff_min', 'Weekdiff_mean',
       'Weekdiff_median', 'Daydiff_max', 'Daydiff_min', 'Daydiff_mean',
       'Daydiff_median', 'have_valid_phone', 'times_valid_phone',
       'total_phone_enter', 'correct_phone_rate', 'num_unique_valid_phone',
       'Loanspaidoff_count', 'Incollection_count',
       'Loanspaidoff_count_in30days', 'Incollection_count_in30days',
       'Loanspaidoff_rate', 'Fraudster_app_count', 'Fraudster_lender_count',
       'Fraudster_app_count_in30days', 'Fraudster_lender_count_in30days',
       'Refused_count', 'Refused_rate', 'Refused_count_within30days',
       'Refused_rate_within30days', 'Refused_count_before30days',
       'Refused_rate_before30days'],
      dtype='object')

#### Now expand the engineering model to whole data repo, which includes all json files

In [15]:
from tqdm import tqdm
fullyworkinglist = []
workingnulllist = []
notworkinglist = []

df_nb = pd.DataFrame([], columns = ['accountnumber', 'frequency', 'avg_amountReq', 'med_amountReq',
       'std_amountReq', 'Yeardiff_max', 'Yeardiff_min', 'Yeardiff_mean',
       'Yeardiff_median', 'Monthdiff_max', 'Monthdiff_min', 'Monthdiff_mean',
       'Monthdiff_median', 'Weekdiff_max', 'Weekdiff_min', 'Weekdiff_mean',
       'Weekdiff_median', 'Daydiff_max', 'Daydiff_min', 'Daydiff_mean',
       'Daydiff_median', 'have_valid_phone', 'times_valid_phone',
       'total_phone_enter', 'correct_phone_rate', 'num_unique_valid_phone',
       'Loanspaidoff_count', 'Incollection_count',
       'Loanspaidoff_count_in30days', 'Incollection_count_in30days',
       'Loanspaidoff_rate', 'Fraudster_app_count', 'Fraudster_lender_count',
       'Fraudster_app_count_in30days', 'Fraudster_lender_count_in30days',
       'Refused_count', 'Refused_rate', 'Refused_count_within30days',
       'Refused_rate_within30days', 'Refused_count_before30days',
       'Refused_rate_before30days'])
for i in tqdm(arr):
    try:
        data_input_path = '../retroData/'+str(i)
        with open(data_input_path) as f:
            data = f.read()
        x = ndmodeling(data)
        x['AppID'] = i[4:-5]
        df_nb = pd.concat([df_nb,x], axis=0)
        if x.MaxHistory.isnull()[0] == True:
            workingnulllist.append(i)
        elif x.MaxHistory.isnull()[0] == False:
            fullyworkinglist.append(i)
    except: 
        notworkinglist.append(i)
        pass

  0%|          | 0/9655 [00:00<?, ?it/s]/var/folders/b7/d1qg7gm9009f_cps7l193hch0000gn/T/ipykernel_11853/2100862969.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_nb = pd.concat([df_nb,x], axis=0)
100%|██████████| 9655/9655 [04:44<00:00, 33.94it/s]


In [18]:
len(fullyworkinglist), len(workingnulllist), len(notworkinglist)

(0, 0, 9655)

In [17]:
df_nb.AppID = df_nb.AppID.astype(int)
df_nb.head()

,accountnumber,frequency,avg_amountReq,med_amountReq,std_amountReq,Yeardiff_max,Yeardiff_min,Yeardiff_mean,Yeardiff_median,Monthdiff_max,...,Fraudster_lender_count,Fraudster_app_count_in30days,Fraudster_lender_count_in30days,Refused_count,Refused_rate,Refused_count_within30days,Refused_rate_within30days,Refused_count_before30days,Refused_rate_before30days,AppID
0,72760,61,683.150943,500.0,409.751585,4,0,2.114754,3.0,10,...,1,0,0,36,0.654545,11,0.785714,25,0.609756,116891
0,47465,2,250.000000,250.0,0.000000,1,0,0.500000,0.5,10,...,0,0,0,0,0.000000,0,0.000000,0,0.000000,65258
0,1888,2,550.000000,550.0,70.710678,2,2,2.000000,2.0,8,...,0,0,0,1,0.500000,1,0.500000,0,0.000000,2152
0,36843,47,980.851064,1000.0,425.102698,5,0,1.872340,1.0,10,...,1,0,0,12,0.315789,1,0.333333,11,0.314286,48875
0,21973,3,1166.666667,1500.0,577.350269,2,1,1.666667,2.0,4,...,0,0,0,1,0.333333,1,1.000000,0,0.000000,28274


In [19]:
print(df_nb.shape)

(7088, 42)


In [24]:
from sqlalchemy import create_engine 
import urllib
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 
USE LF_LMSMASTER
-- From Anson
drop table if EXISTS #t1
select A.Application_ID, A.PortfolioID, A.CustomerID, A.ApplicationDate,
case when ApplicationSteps like '%S%' then 1 else 0 end as NewlyScored,
case when A.ApplicationStatus in ('A','P') then 1 else 0 end as Accepted,
case when A.ApplicationStatus in ('A','P') then A.LeadPurchasePrice else 0 end as LeadPurchasePrice,
case when L.LoanStatus not in ('V','W','G','K') then 1 else 0 end as Originated,
L.LoanID, datediff(year, VW.DOB, A.ApplicationDate) as Age,
case when VW.Frequency in ('B','S') then 'B' else VW.Frequency end as Frequency,
case when L.LoanStatus not in ('V','W','G','K') then L.OriginatedAmount else null end as OriginatedAmount,
LP.Provider_name 
into #t1
from Application A
left join Loans L on A.Application_ID = L.ApplicationID and A.PortFolioID = L.PortFolioID
left join LeadProvider LP on A.LeadProviderID = LP.LeadProviderID
left join LF_LMS_Logs..VW_ApplicationDump VW on A.APPGUID = VW.APPGUID
where A.ApplicationDate>'2023-04-30' and datediff(week, A.ApplicationDate, getdate()) >2 
-- and A.ApplicationSteps not like '%R%' and A.ApplicationSteps not like '%O%' 
order by A.ApplicationDate asc


-- scoring result for PlaidUDW_v1 and NegativeDBModelLP_v1
drop table if EXISTS #t2
select *
into #t2 from
(select A.*, 
cast(substring(B1.ExtResponse, CHARINDEX('IBVBand',B1.ExtResponse)+9,1) as int) as NDBand,
cast(substring(B2.ExtResponse, CHARINDEX('NDBand',B2.ExtResponse)+8,1) as int) as IBVBand,
row_number() over (partition by A.Application_ID, A.PortfolioID order by B.ApplicationDate desc) as RN
from #t1 A 
left join Application B on A.CustomerID = B.CustomerID and A.ApplicationDate >= B.ApplicationDate and B.ApplicationStatus like '%S%'
left join ScoringPythonResult B1 on B.Application_ID = B1.ApplicationID and B.PortFolioID = B1.PortfolioID and B1.iLabel= 'IBVBand' 
left join ScoringPythonResult B2 on B.Application_ID = B2.ApplicationID and B.PortFolioID = B2.PortfolioID and B2.iLabel= 'NDBand' ) K where RN=1



ALTER TABLE #t2
DROP COLUMN RN;


DROP TABLE IF EXISTS #LF_LoanDefault
SELECT L.LoanID, L.ApplicationID AS Application_ID, A.APPGUID, A.ApplicationDate, A.ApplicationSteps, L.PortFolioID, L.LoanStatus,
P.InstallmentNumber, P.PaymentStatus, P.PaymentType, P.PaymentMode, P.AttemptNo, P.TransactionDate, P.PaymentID,
I.InstallmentID, I.iPaymentMode, I.DueDate, I.Status, -- used to exclude pendings (code 684)
(CASE WHEN I.Status=684 THEN 1 ELSE 0 END) AS Pending
INTO #LF_LoanDefault
FROM LF_LMSMaster..Loans L
LEFT JOIN LF_LMSMaster..Payment P ON P.LoanID = L.LoanID
LEFT JOIN LF_LMSMaster..Installments I ON I.InstallmentID = P.InstallmentID
LEFT JOIN LF_LMSMaster..Application A ON A.PortfolioID=L.PortfolioID AND A.Application_ID = L.ApplicationID
WHERE A.ApplicationDate>'2023-04-30' AND datediff(week, A.ApplicationDate, getdate()) >2 
AND I.InstallmentNumber = 1

DROP TABLE IF EXISTS #LoanDefault_Flag
SELECT 
    L.*,
    -- FPDFA flag
    CASE 
        WHEN L.PaymentStatus = 'R'
             AND L.PaymentType IN ('I','S','A')
             AND L.PaymentMode IN ('A','B','D')
             AND L.DueDate <= CAST(GETDATE() AS date)
             AND NOT EXISTS (
                 SELECT 1
                 FROM #LF_LoanDefault ld
                 WHERE ld.InstallmentID = L.InstallmentID
                   AND ld.PaymentStatus = 'D'
                   AND ld.PaymentType NOT IN ('3','~','Q')
                   AND ld.PaymentMode IN ('A','D','B')
                   AND CONVERT(date, ld.TransactionDate) = CONVERT(date, L.DueDate)
             )
        THEN 1 ELSE 0 END AS is_FPDFA,
        CASE 
        WHEN L.LoanStatus NOT IN ('V','W','G','K')
             AND NOT (
                 L.iPaymentMode = 144 
                 AND L.Pending = 1
                 AND L.DueDate >= CAST(GETDATE() AS date)
             )
        THEN 1 ELSE 0 
    END AS is_loan_first_install
INTO #LoanDefault_Flag
FROM #LF_LoanDefault L;


DROP TABLE IF EXISTS #LoanDefault_Dedup;
WITH dedup AS (
    SELECT LoanID, Application_ID, APPGUID, ApplicationDate, ApplicationSteps, PortfolioID, LoanStatus,
           InstallmentNumber, PaymentStatus, PaymentType, PaymentMode, AttemptNo, TransactionDate,
           PaymentID, InstallmentID, iPaymentMode, DueDate, Status, Pending, is_FPDFA, is_loan_first_install,
           ROW_NUMBER() OVER (PARTITION BY Application_ID, PortfolioID ORDER BY is_FPDFA DESC) AS rn
    FROM #LoanDefault_Flag
)
SELECT LoanID, Application_ID, APPGUID, ApplicationDate, ApplicationSteps, PortfolioID, LoanStatus,
       InstallmentNumber, PaymentStatus, PaymentType, PaymentMode, AttemptNo, TransactionDate,
       PaymentID, InstallmentID, iPaymentMode, DueDate, Status, Pending, is_FPDFA, is_loan_first_install
INTO #LoanDefault_Dedup
FROM dedup
WHERE rn = 1;
-- Check Top 100
SELECT TOP 10 *
FROM #LoanDefault_Dedup


drop table if EXISTS #t3
select A.*, B.is_FPDFA, B.PaymentStatus, B.LoanStatus
into #t3
from #t2 A 
inner join #LoanDefault_Dedup B on A.LoanID = B.LoanID


drop table if EXISTS #t31
select Application_ID, PortFolioID, LoanID, (1-max(FstInstPaidOff)) as FPDAA
into #t31 from
(select A.*, case when P.PaymentStatus = 'D' then 1 else 0 end as FstInstPaidOff, P.PaymentStatus
from #t2 A 
inner join Payment P on A.LoanID = P.LoanID and P.PaymentMode IN ('A','D','B') and PaymentType NOT IN ('3','~','Q')
                        and P.InstallmentNumber=1 and P.AttemptNo >= 1 and P.PaymentDate <= getdate() and P.PaymentStatus in ('D','R','S','B')
where A.Originated=1) K group by Application_ID, PortFolioID, LoanID


drop table if EXISTS #t4
select A.Application_ID, A.PortFolioID, A.LoanID, sum(case when P.PaymentStatus = 'D' then P.PaymentAmount else 0 end) as PaidOffPaymentAmount,
sum(case when P.PaymentStatus = 'R' then 1 else 0 end) as PmtReturn, count(*) as PmtCount
into #t4
from #t2 A 
inner join Payment P on A.LoanID = P.LoanID and P.PaymentMode not in ('V','P','T','H','G') and PaymentType in ('I','S','Z','A','X','Q') 
                        and P.InstallmentNumber >= 1 and P.PaymentDate <= getdate() and P.PaymentStatus in ('D','R') and P.PaymentAmount>=2.95 and (P.PaymentAmount > P.FeeChargePaid)
where A.Originated=1
group by A.Application_ID, A.PortFolioID, A.LoanID
"""
query2 = """
select A.*,B.is_FPDFA, B2.FPDAA, C.PaidOffPaymentAmount, C.PmtCount, C.PmtReturn,
C.PaidOffPaymentAmount / (A.OriginatedAmount+0.0) as Payin,
case when C.PaidOffPaymentAmount >= A.OriginatedAmount * 1.05 then 1 else 0 end as IsGood
from #t2 A  -- avg(OriginatedAmount+0.0) as AvgLoanAmount, sum(FPDFA) as FPDFA, sum(FPDFA+0.0)/count(*)*100 as FPDFARate
left join #t3 B on A.Application_ID = B.Application_ID and A.PortFolioID = B.PortFolioID
left join #t31 B2 on A.Application_ID = B2.Application_ID and A.PortFolioID = B2.PortFolioID
left join #t4 C on A.Application_ID = C.Application_ID and A.PortFolioID = C.PortFolioID
where A.Originated = 1 and ((datediff(day, A.ApplicationDate, getdate()) >= 110) or (C.PaidOffPaymentAmount > A.OriginatedAmount))
"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_perf = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')
df_perf = df_perf.rename(columns = {'Application_ID':'AppID'})
df_perf.AppID = df_perf.AppID.astype(int)
df_perf.head(3)

,AppID,PortfolioID,CustomerID,ApplicationDate,NewlyScored,Accepted,LeadPurchasePrice,Originated,LoanID,Age,...,Provider_name,NDBand,IBVBand,is_FPDFA,FPDAA,PaidOffPaymentAmount,PmtCount,PmtReturn,Payin,IsGood
0,172,1,132,2023-05-01 09:48:10,1,1,5.0,1,I49-0,35.0,...,Dot818,None,None,0.0,0.0,69.2,6.0,5.0,0.2768,0
1,174,1,134,2023-05-01 10:14:45,1,1,5.0,1,I50-0,51.0,...,Dot818,None,None,0.0,0.0,69.2,6.0,5.0,0.2768,0
2,197,1,153,2023-05-01 14:08:01,1,1,5.0,1,I60-0,27.0,...,Dot818,None,None,1.0,1.0,346.0,7.0,2.0,1.3840,1


In [26]:
# For FPD Modeling
df_final = df_perf[['AppID','CustomerID','ApplicationDate','Age']].merge(df_fe, on=['AppID'],how='inner')
# df_final = df_final.rename(columns = {'AppID':'ScoredAppID','ApplicationDate':"ScoredAppDate"})
df_final = df_final.merge(df_perf[df_perf.FPDAA.notnull()][['IsGood','AppID','Payin','FPDAA','is_FPDFA']], how='inner', on=['AppID'])
# len(df_final[(df_final.FPDAA.notnull())&(df_final.MaxHistory.notnull())]) # 1k makes sense

In [27]:
df_final[(df_final.IsGood.notnull())].to_csv('ibv_input_data.csv',index=False)

In [31]:
df_final.head()

,AppID,CustomerID,ApplicationDate,Age,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,...,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,NUM_EI_PMT,AMT_EI_PMT,NUM_Bree,IsGood,Payin,FPDAA,is_FPDFA
0,172,132,2023-05-01 09:48:10,35.0,63,1.023761,0.866899,225.379417,225.313083,189.683333,...,0,0,0.0,305.0,30387.57,287.0,0,0.2768,0.0,0.0
1,174,134,2023-05-01 10:14:45,51.0,90,1.194366,1.061650,195.728267,205.952200,124.399500,...,0,0,0.0,567.0,30892.83,458.0,0,0.2768,0.0,0.0
2,197,153,2023-05-01 14:08:01,27.0,60,1.027346,0.047099,239.111111,239.341778,95.763167,...,0,0,0.0,236.0,21540.76,221.0,1,1.3840,1.0,1.0
3,308,244,2023-05-04 10:52:57,39.0,90,1.100375,0.236295,230.337083,256.043333,156.048250,...,0,0,0.0,520.0,30725.47,500.0,0,0.9688,0.0,0.0
4,340,273,2023-05-04 14:48:10,26.0,88,0.832004,0.365121,104.749083,104.939917,9.995444,...,3,61,466.0,242.0,12592.79,224.0,0,0.6559,0.0,0.0


In [28]:
print(len(df_final[(df_final.IsGood.notnull())&(df_final.MaxHistory.isnull())])) # without model step 1
print(len(df_final[(df_final.IsGood.notnull())&(df_final.MaxHistory.notnull())])) # with model step 2

0
7968
